In [1]:
import pandas as pd
import datetime as dt
import pymongo
import re
import csv

In [6]:
source_data = 'Yahoo Finance'
stocks_df = pd.read_csv('AMZN NASDAQ Yahoo Finance.csv')
stocks_df.head(5)

,Date,Open,High,Low,Price,Vol.,Change %,Exchange,Country,Currency
0,2/21/2018,1485.000000,1503.489990,1478.920044,1482.920044,6304400,0.99%,NASDAQ,USA,USD
1,2/22/2018,1495.359985,1502.540039,1475.760010,1485.339966,4858100,0.16%,NASDAQ,USA,USD
2,2/23/2018,1495.339966,1500.000000,1486.500000,1500.000000,4418100,0.99%,NASDAQ,USA,USD
3,2/26/2018,1509.199951,1522.839966,1507.000000,1521.949951,4955000,1.46%,NASDAQ,USA,USD
4,2/27/2018,1524.500000,1526.780029,1507.209961,1511.979980,4808800,-0.66%,NASDAQ,USA,USD


In [7]:
stocks_df.dtypes



Date         object
Open        float64
High        float64
Low         float64
Price       float64
Vol.          int64
Change %     object
Exchange     object
Country      object
Currency     object
dtype: object

In [8]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])
stocks_df['Price'] = stocks_df['Price'].astype(float)
stocks_df['Open'] = stocks_df['Open'].astype(float)
stocks_df['High'] = stocks_df['High'].astype(float)
stocks_df['Low'] = stocks_df['Low'].astype(float)
stocks_df['Change %'] = pd.to_numeric(stocks_df['Change %'].str.replace('%', ''))
stocks_df['Exchange'] = stocks_df['Exchange'].apply(str)
stocks_df['Country'] = stocks_df['Country'].apply(str)
stocks_df['Currency'] = stocks_df['Currency'].apply(str)

In [9]:
stocks_df.dtypes

Date        datetime64[ns]
Open               float64
High               float64
Low                float64
Price              float64
Vol.                 int64
Change %           float64
Exchange            object
Country             object
Currency            object
dtype: object

In [230]:
#volume conversion function for K and M 
def convertVolume(data):
    temp = 0
    if re.search('M', data):
        temp = float(data.replace('M', '')) * 1000000
    elif re.search('K', data):
        temp = float(data.replace('K', '')) * 1000
        
    return temp

In [10]:
stocks_df['Vol.'] = stocks_df['Vol.'].astype(int)


In [11]:
stocks_df['Low'].head()


0    1478.920044
1    1475.760010
2    1486.500000
3    1507.000000
4    1507.209961
Name: Low, dtype: float64

In [12]:
#insert data to database

In [13]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [16]:
dbnames = client.list_database_names()
if 'Stocks' in dbnames:
    print ("It's there!")
    db = client.Stocks
else:
    # Define the class for stocks in database Mongo
    print('Creating Database')
    db = client['Stocks']

# Query all students
# Here, db.students refers to the collection 'classroom '
#historical_data = db.historical_data.find()
#print(historical_data)
# Iterate through each student in the collection
for index, row in stocks_df.iterrows():
    db.historical_data.insert_one(
    {
        'Date': row['Date'],
        'Price': row['Price'],
        'Open': row['Open'],
        'High': row['High'],
        'Low': row['Low'],
        'Vol': row['Vol.'],
        'Change': row['Change %'],
        'Exchange': row['Exchange'],
        'Country': row['Country'],
        'Currency': row['Currency'],
        'Source': source_data
    }
)
print('Done!')


It's there!
Done!
